#  **Qwen 2.5-7B com raciocínio (2º Tentativa)**

**Authors:** Renato Freitas Bessa, Adonias Caetano de Oliveira, Ariel Soares Teles

**Institution:** UFDPar - PPGBiotec

**Objective:** responde as questões sem respostas dos exames do revalida com Qwen 2.5 por meio de cadeia de raciocínio (2º tentativa).

In [ ]:
from google.colab import drive

# Montar o drive do Google
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install -U bitsandbytes

In [ ]:
import pandas as pd
import re
import random

## **Loading data**

In [ ]:
#sheet_names = ["2024.01"]
sheet_names = ["2017.01", "2017.02", "2020.01", "2020.02", "2021.01", "2022.01", "2022.02", "2023.01", "2023.02", "2024.01"]

In [ ]:
url = '<public link to access the questions from the 2017.01 to 2024.01 exams>'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
provas = pd.read_excel(read_url)
provas.head()

,Perguntas-2017.01,Gabarito-2017.01,Perguntas-2017.02,Gabarito-2017.02,Perguntas-2020.01,Gabarito-2020.01,Perguntas-2020.02,Gabarito-2020.02,Perguntas-2021.01,Gabarito-2021.01,Perguntas-2022.01,Gabarito-2022.01,Perguntas-2022.02,Gabarito-2022.02,Perguntas-2023.01,Gabarito-2023.01,Perguntas-2023.02,Gabarito-2023.02,Perguntas-2024.01,Gabarito-2024.01
0,1.\tUma mulher com 23 anos de idade comparece ...,NULA,1.\tUma mulher com 38 anos de idade procurou a...,C,1.\tAo atender uma primigesta com 15 anos de i...,A,"1.\tUma mulher com 43 anos de idade, com 80 kg...",C,"1.\tPrimigesta, de 25 anos de idade, com 34 se...",B,"1.\tUma mulher com 51 anos de idade, em tratam...",NULA,1.\tIMAGEM,C,"1.\tUm paciente com 54 anos, em acompanhamento...",D,"1.\tUma mulher de 60 anos, hipertensa, em uso ...",B,"1 Uma mulher com 25 anos, solteira, sem comorb...",A
1,2.\tUm homem com 65 anos de idade encontra-se ...,B,2.\tUma mulher com 23 anos de idade comparece ...,NULA,2.\tUma mulher com 35 anos de idade exerce pro...,C,2.\tAo atender uma primigesta com 15 anos de i...,A,2.\tUm recém-nascido com 20 dias de vida dá en...,C,"2.\tUm paciente com 72 anos de idade, tabagist...",C,"2.\tHomem de 75 anos de idade, tabagista 1 maç...",A,"2.\tUm homem com 62 anos comparece, acompanhad...",D,2.\tUm paciente de 48 anos é internado devido ...,D,"2 Um paciente com 28 anos, vítima de atropelam...",A
2,3.\tUma mulher com 38 anos de idade procurou a...,C,3.\tUm homem com 65 anos de idade encontra-se ...,B,"3.\tUma mulher com 43 anos de idade, com 80 kg...",C,3.\tUma mulher com 35 anos de idade exerce pro...,C,"3.\t Um homem, de 37 anos de idade, com AIDS/H...",B,"3.\tUm menino com 5 anos de idade, acompanhado...",C,3.\tLactente de 6 meses está sendo atendido na...,B,"3.\tUma lactente com 6 meses, nascida a termo,...",B,"3.\tUm menino de 9 anos, com peso de 29 kg e a...",D,3 Um lactente com 3 meses comparece a uma cons...,C
3,4.\tIMAGEM,IMAGEM,"4.\tUma mulher com 32 anos de idade, no quinto...",B,4.\tUma mulher com 18 anos de idade vai ao amb...,A,4.\tUma mulher com 48 anos de idade vai a uma ...,D,4.\tPaciente teve parto cesárea há 8 dias e re...,D,4.\tSegundo o Instituto Nacional do Câncer (IN...,D,"4.\tAdolescente de 14 anos de idade, primípara...",D,"4.\tUma paciente com 21 anos, nuligesta, compa...",A,4.\tUma mulher de 49 anos comparece à unidade ...,B,"4 Uma gestante com 24 anos, G2 P1 A0, chega à ...",D
4,"5.\tUma criança com 4 anos de idade, cujos pai...",A,5.\tIMAGEM,B,5.\tUma criança com 6 anos de idade é levada p...,B,5.\tUma mulher com 18 anos de idade vai ao amb...,A,"5.\tPaciente de 68 anos de idade, do sexo masc...",B,"5.\tUm homem com 48 anos de idade é obeso, tab...",A,5.\tUma mulher de 34 anos com diagnóstico de d...,D,5.\tSindemia é um conjunto de problemas de saú...,A,5.\tEm um território com população em situação...,B,5 Ao assumir a coordenação de uma equipe de sa...,A


In [ ]:
# read the data
respostas_geradas = pd.read_excel("resposta_gabarito.xlsx")
respostas_geradas.head()

,questao,respostas_2017.01,gabarito_2017.01,respostas_2017.02,gabarito_2017.02,respostas_2020.01,gabarito_2020.01,respostas_2020.02,gabarito_2020.02,respostas_2021.01,...,respostas_2022.01,gabarito_2022.01,respostas_2022.02,gabarito_2022.02,respostas_2023.01,gabarito_2023.01,respostas_2023.02,gabarito_2023.02,respostas_2024.01,gabarito_2024.01
0,1,NULA,NULA,VOID,C,VOID,A,VOID,C,VOID,...,NULA,NULA,VOID,C,VOID,D,D,B,VOID,A
1,2,B,B,NULA,NULA,C,C,VOID,A,VOID,...,VOID,C,VOID,A,VOID,D,VOID,D,D,A
2,3,A,C,VOID,B,VOID,C,VOID,C,VOID,...,D,C,C,B,VOID,B,VOID,D,VOID,C
3,4,IMAGEM,IMAGEM,VOID,B,VOID,A,VOID,D,VOID,...,C,D,A,D,VOID,A,VOID,B,C,D
4,5,A,A,VOID,B,VOID,B,VOID,A,VOID,...,VOID,A,VOID,D,VOID,A,VOID,B,VOID,A


## **Installation**

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### **Unsloth**

In [ ]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.


    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.11 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
checkpoint_modelo = "unsloth/Qwen2.5-7B"

#"unsloth/Qwen2-7B-Instruct-bnb-4bit"
# "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit" <=
#'unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit'
# "unsloth/llama-3-8b-Instruct-bnb-4bit"
# "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
# "unsloth/Qwen2.5-14B-Instruct-bnb-4bit"
# "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
# "unsloth/Qwen2-1.5B-Instruct-bnb-4bit",
# "unsloth/Qwen2-7B-Instruct-bnb-4bit",
#    "unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit",
#    "unsloth/Qwen2.5-1.5B-Instruct-bnb-4bit",
#    "unsloth/Qwen2.5-3B-Instruct-bnb-4bit",
#    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
#    "unsloth/Qwen2.5-14B-Instruct-bnb-4bit",
#    "unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
#    "unsloth/Phi-3-medium-4k-instruct-bnb-4bit",


In [ ]:
modelo, tokenizador = FastLanguageModel.from_pretrained(
    model_name = checkpoint_modelo,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
'''
lista = provas['Perguntas-' + '2017.02'].to_list()
pergunta = lista[41]
prompt = "Dado a questão a seguir, qual item está correto? A ou B ou C ou D? Responda apenas com a letra e sem explicação da resposta. Questão: " + pergunta
prompt
prompt_tokenizado = tokenizador([prompt], return_tensors='pt').to('cuda')
FastLanguageModel.for_inference(modelo)
output = modelo.generate(**prompt_tokenizado, max_new_tokens=2000)
resposta_ids = output[0]
resposta_texto = tokenizador.decode(resposta_ids, skip_special_tokens=True)
resposta_texto
'''

'\nlista = provas[\'Perguntas-\' + \'2017.02\'].to_list()\npergunta = lista[41]\nprompt = "Dado a questão a seguir, qual item está correto? A ou B ou C ou D? Responda apenas com a letra e sem explicação da resposta. Questão: " + pergunta\nprompt\nprompt_tokenizado = tokenizador([prompt], return_tensors=\'pt\').to(\'cuda\')\nFastLanguageModel.for_inference(modelo)\noutput = modelo.generate(**prompt_tokenizado, max_new_tokens=2000)\nresposta_ids = output[0]\nresposta_texto = tokenizador.decode(resposta_ids, skip_special_tokens=True)\nresposta_texto\n'

In [ ]:
MAX_TOKENS = 10 # tamanho de tokens da saída
respostas_llm = {}
for name in sheet_names:
  perguntas = provas['Perguntas-' + name].to_list()
  gabaritos = provas['Gabarito-' + name].to_list()
  respostas = respostas_geradas['respostas_' + name].to_list()
  respostas_llm['Repostas-' + name] = []

  questao = 0

  for pergunta, gabarito, resposta in zip(perguntas, gabaritos, respostas):

    questao += 1

    if resposta == "VOID" and gabarito != "IMAGEM" and gabarito != "TABELA" and gabarito != "NULA":

      #prompt = "Dado a questão a seguir, qual item está correto? A ou B ou C ou D? Responda apenas com a letra e sem explicação da resposta. Questão: " + pergunta
      prompt = "Considere a questão: " + pergunta + ". O item correto é qual letra?"
      prompt_tokenizado = tokenizador([prompt], return_tensors='pt').to('cuda')
      FastLanguageModel.for_inference(modelo)
      output = modelo.generate(**prompt_tokenizado, max_new_tokens=MAX_TOKENS)
      resposta_ids = output[0]
      resposta_texto = tokenizador.decode(resposta_ids, skip_special_tokens=True)

      pattern = r"resposta correta é [a-d]|resposta é [a-d]|Resposta: [a-d]|Resposta correta: [a-d]|alternativa correta é [a-d]|correta é [a-d]|correta é a letra [a-d]|correcta é [a-d]|correcta é a letra [a-d]|A resposta [a-d]|resposta correta seria [a-d]|é a \"\"[a-d]\)|seria \"\"[a-d]\"\"|a resposta mais comum seria \'[a-d]\'|resposta correta é: [a-d]"

      # Verifica se há correspondência
      match = re.search(pattern, resposta_texto, re.IGNORECASE)

      if match:
        letra = match.group(0)[-1].upper()
        respostas_llm['Repostas-' + name].append(letra)
        print(f"{letra}")
      elif (match := re.search(r"a opção \"\"[a-d]\"\" parece ser a mais adequada", resposta, re.IGNORECASE)):
        string = match.group(0)
        match = re.search(r"a opção \"\"[a-d]", resposta, re.IGNORECASE)
        letra = match.group(0)[-1].upper()
        respostas['Repostas-' + name].append(letra)
        print(f"{letra}")
      else:
        respostas_llm['Repostas-' + name].append("ALUCINOU")
        print("Não encontrado na questão " + str(questao))

    else:
      respostas_llm['Repostas-' + name].append(" ")



A
Não encontrado na questão 12
Não encontrado na questão 13
Não encontrado na questão 14
C
Não encontrado na questão 16
Não encontrado na questão 22
Não encontrado na questão 23
Não encontrado na questão 26
Não encontrado na questão 27
Não encontrado na questão 28
C
A
Não encontrado na questão 32
Não encontrado na questão 33
A
C
A
Não encontrado na questão 44
Não encontrado na questão 46
Não encontrado na questão 49
Não encontrado na questão 50
Não encontrado na questão 55
C
Não encontrado na questão 58
Não encontrado na questão 62
A
C
Não encontrado na questão 68
Não encontrado na questão 73
Não encontrado na questão 74
Não encontrado na questão 77
Não encontrado na questão 78
Não encontrado na questão 80
D
Não encontrado na questão 83
Não encontrado na questão 88
Não encontrado na questão 89
C
Não encontrado na questão 91
C
C
C
Não encontrado na questão 99
Não encontrado na questão 100
Não encontrado na questão 1
Não encontrado na questão 3
Não encontrado na questão 4
Não encontrado 

In [ ]:
from google.colab import files

df = pd.DataFrame(respostas_llm)
df.to_excel('respostas.xlsx', index=False)

# Especificar o caminho do arquivo no Google Drive
caminho_arquivo = '/content/drive/My Drive/respostas.xlsx'

df.to_excel(caminho_arquivo, index=False)

files.download('respostas.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>